# Bibliotecas

In [ ]:
import pandas as pd #análise de dados em tabelas
import numpy as np #análise de números
import seaborn as sb #análise em gráficos

# Cardiovascular Machine Learning Classification Model
With categorical features

In [ ]:
cvd = pd.read_csv('http://bit.ly/cardiodados')

In [ ]:
cvd.columns.tolist()

In [ ]:
cvd[['htn', 'treat']]

In [ ]:
cvd_Shtn = cvd[cvd['htn'] == 'Y']
cvd_Nhtn = cvd[cvd['htn'] == 'N']

In [ ]:
total_cvd_htnS = len(cvd_Shtn)
total_cvd_htnN = len(cvd_Nhtn)
total_cvd_htnT = len(cvd_Shtn) + len(cvd_Nhtn)

In [ ]:
total_cvd_htnS / total_cvd_htnT

In [ ]:
cvd.groupby('htn')['treat'].value_counts()

In [ ]:
cvd['htn'] == 'Y'

In [ ]:
cvd.head()

In [ ]:
cvd.shape

In [ ]:
cvd_col_imc = cvd[(cvd['tchol'] > 200) & (cvd['bmi'] > 30)]

In [ ]:
problema = cvd[(cvd['htn'] == 'Y') | (cvd['t2d'] =='Y') | (cvd['numAge'] > 35)]

In [ ]:
problema.to_csv('e.csv')

## .loc

In [ ]:

cvd.loc[cvd['htn'] == 'N', 'htn_bin'] = 0
cvd['htn_bin'].value_counts()

In [ ]:
cvd['race'].value_counts()

In [ ]:
cvd.loc[cvd['race'] == 'White', 'race_W'] = 1
cvd.loc[cvd['race'] != 'White', 'race_W'] = 0

cvd.loc[cvd['race'] == 'Asian/PI', 'race_As'] = 1
cvd.loc[cvd['race'] != 'Asian/PI', 'race_As'] = 0

cvd.loc[cvd['race'] == 'Black/AfAm', 'race_Af'] = 1
cvd.loc[cvd['race'] != 'Black/AfAm', 'race_Af'] = 0

cvd.loc[cvd['race'] == 'AmInd', 'race_Am'] = 1
cvd.loc[cvd['race'] != 'AmInd', 'race_Am'] = 0

cvd['race_As'].value_counts()

In [ ]:
cvd.loc[cvd['bmi'] > 30, 'bmi_bin'] = 1
cvd.loc[cvd['bmi'] <= 30, 'bmi_bin'] = 0

In [ ]:
cvd.head()

### Transform Data

In [ ]:
def transform(valor):
  if valor == 'Y':
    return 1
  else:
    return 0

In [ ]:
def gender(valor):
  if valor == 'F':
    return 1
  else:
    return 0

In [ ]:
cvd['htn'] = cvd['htn'].map(transform) 
cvd['treat'] = cvd['treat'].map(transform) 
cvd['smoking'] = cvd['smoking'].map(transform) 
cvd['t2d'] = cvd['t2d'].map(transform) 
cvd['cvd'] = cvd['cvd'].map(transform) 
cvd['gender'] = cvd['gender'].map(gender) 

In [ ]:
cvd.race.value_counts()

In [ ]:
cvd['asian'] = 0
cvd.loc[ cvd['race'] == 'Asian/PI', 'asian' ] = 1

cvd['white'] = 0
cvd.loc[ cvd['race'] == 'White', 'asian' ] = 1

cvd['black'] = 0
cvd.loc[ cvd['race'] == 'Black/AfAm', 'black' ] = 1

cvd['amind'] = 0
cvd.loc[ cvd['race'] == 'AmInd', 'asian' ] = 1

In [ ]:
cvd = cvd.drop('race', 1)

### Data types

In [ ]:
cvd.head()

age is categorical or numerical?

In [ ]:
cvd.age.value_counts()

### Prepare Train Data

In [ ]:
cvd.head()

In [ ]:
cvd.columns

In [ ]:
cvd = cvd.dropna()

In [ ]:
X = cvd[['htn', 'treat', 'smoking', 't2d', 'gender',
       'numAge', 'bmi', 'tchol', 'sbp', 'htn_bin', 'race_W', 'race_As',
       'race_Af', 'race_Am', 'bmi_bin', 'asian', 'white', 'black', 'amind']]
y = cvd.cvd

### Split Train Test

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0) 

### Prepare model, train, predict and evaluate

In [ ]:
!pip install xgboost

In [ ]:
# Import models
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
clf1 = LogisticRegression() # regressão logistica
clf2 = RandomForestClassifier() # arvores aleatorias
clf3 = xgb.XGBClassifier() # boost gradient
clf4 = SVC(gamma='auto')
clf5 = KNeighborsClassifier(n_neighbors=3)

# Train, Predict and Evaluate the model 
from sklearn.metrics import accuracy_score, classification_report, recall_score
from sklearn.model_selection import cross_val_score
for clf, label in zip([clf1, clf2, clf3, clf4, clf5], 
                      ['Logistic Regression', 'Random Forest', 'XGBoost', 'SVM', 'KNN']):
    clf.fit(X_train, y_train) #training model
    p = clf.predict(X_test) #predicting model
    scores = cross_val_score(clf, X_test, y_test, cv=10, scoring='recall_macro')
    print(label)
    print(classification_report(y_test, p)) #evaluating model
    print('Sensibilidade =', recall_score(y_test, p))
    print('Sensibilidade da Validação Cruzada', np.mean(scores))
    print("_______________________________________________________")

In [ ]:
p

In [ ]:
sub = pd.Series(p, index=cvd_test['patientID'], name='cvd_bin')

In [ ]:
sub.to_csv('primeiro.csv', header=True)

In [ ]:
!head -n10 primeiro.csv